In [44]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
print("cwd:", os.getcwd())
print("files:", sorted(os.listdir(".")))

cwd: /Users/riddhirajput/Documents/Riddhi/Think School/scraping
files: ['.ipynb_checkpoints', 'Think_School.ipynb', 'config.py']


In [6]:
from googleapiclient.discovery import build
import pandas as pd
from config import API_KEY

api_key = API_KEY
channel_id = "UCYpRDnhk5H8hEZ8i9iJPFyA"

youtube = build('youtube', 'v3', developerKey=api_key)

def get_videos(youtube, channel_id):
    videos = []
    next_page_token = None

    while True:
        res = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=50,
            order="date",
            pageToken=next_page_token
        ).execute()

        for item in res['items']:
            if item['id']['kind'] == 'youtube#video':
                video = {
                    'video_id': item['id']['videoId'],
                    'title': item['snippet']['title'],
                    'published_at': item['snippet']['publishedAt'],
                }
                videos.append(video)

        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break

    return videos

videos = get_videos(youtube, channel_id)
df = pd.DataFrame(videos)
print("Videos Collected:", len(df))

df.to_csv("thinkschool_videos.csv", index=False)


Videos Collected: 0


In [7]:
from googleapiclient.discovery import build
import pandas as pd
from config import API_KEY

api_key = API_KEY
channel_id = "UCKZozRVHRYsYHGEyNKuhhdA"

youtube = build('youtube', 'v3', developerKey=api_key)

# Step 1: Get Uploads Playlist ID
res = youtube.channels().list(
    id=channel_id,
    part='contentDetails'
).execute()

playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
print("Uploads Playlist ID:", playlist_id)

# Step 2: Get Videos from the Playlist
videos = []
next_page_token = None

while True:
    pl_request = youtube.playlistItems().list(
        part='snippet',
        playlistId=playlist_id,
        maxResults=50,
        pageToken=next_page_token
    )
    pl_response = pl_request.execute()

    for item in pl_response['items']:
        video = {
            'video_id': item['snippet']['resourceId']['videoId'],
            'title': item['snippet']['title'],
            'published_at': item['snippet']['publishedAt']
        }
        videos.append(video)

    next_page_token = pl_response.get('nextPageToken')

    if not next_page_token:
        break

df = pd.DataFrame(videos)
print("Videos Collected:", len(df))
df.to_csv("thinkschool_videos.csv", index=False)


Uploads Playlist ID: UUKZozRVHRYsYHGEyNKuhhdA
Videos Collected: 830


In [8]:
from googleapiclient.discovery import build
from config import API_KEY

api_key = API_KEY
youtube = build('youtube', 'v3', developerKey=api_key)

# Search for the channel by name
res = youtube.search().list(
    q="Think School",
    part="snippet",
    type="channel",
    maxResults=1
).execute()

print(res['items'][0]['snippet']['title'])
print("CHANNEL ID:", res['items'][0]['id']['channelId'])


Think School
CHANNEL ID: UCKZozRVHRYsYHGEyNKuhhdA


In [10]:
def get_video_stats(youtube, df):
    stats = []
    for i in range(0, len(df), 50):
        ids = ','.join(df['video_id'][i:i+50])
        res = youtube.videos().list(
            part="statistics,contentDetails",
            id=ids
        ).execute()

        for item in res['items']:
            stats.append({
                'video_id': item['id'],
                'view_count': int(item['statistics'].get('viewCount', 0)),
                'like_count': int(item['statistics'].get('likeCount', 0)),
                'comment_count': int(item['statistics'].get('commentCount', 0)),
                'duration': item['contentDetails']['duration']
            })
    return stats

stats = get_video_stats(youtube, df)
df_stats = pd.DataFrame(stats)
merged = pd.merge(df, df_stats, on='video_id', how='left')
merged.to_csv("../data/thinkschool_videos_full.csv", index=False)
print("Data Saved ✅")

Data Saved ✅


In [11]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


### Cleaning and Reviewing the data

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [15]:
df = pd.read_csv('../data/thinkschool_videos_full.csv')

In [16]:
df.head()

,video_id,title,published_at,view_count,like_count,comment_count,duration
0,X0QmjGzoW98,SHOCKING : Meesho’s Hidden Superpower has made...,2025-12-03T04:53:52Z,48511,2910,126,PT14M48S
1,UNPqb4ThKHg,TESLA vs Uber : Will UBER Be Dead by 2030? | B...,2025-11-27T12:08:26Z,203724,5828,329,PT17M45S
2,lBCiFPh87ik,Packaging You Can't Ignore!,2025-11-27T06:50:02Z,345980,8398,48,PT37S
3,-XLYBiMJDI0,How can India go global?,2025-11-26T11:30:00Z,34283,636,8,PT52S
4,XczYinm2pQc,Smart Founders Don’t Ask… They Observe,2025-11-25T12:45:11Z,34146,585,2,PT37S


In [17]:
df.shape

(830, 7)

In [18]:
df.describe()

,view_count,like_count,comment_count
count,8.300000e+02,830.000000,830.000000
mean,6.506577e+05,22164.368675,917.721687
std,1.037969e+06,39953.994086,1629.150151
min,0.000000e+00,0.000000,0.000000
25%,7.890300e+04,2240.500000,18.000000
50%,3.469725e+05,12361.000000,264.500000
75%,8.968480e+05,30455.250000,1188.000000
max,1.458610e+07,717795.000000,23122.000000


In [19]:
df.info

<bound method DataFrame.info of         video_id                                              title  \
0    X0QmjGzoW98  SHOCKING : Meesho’s Hidden Superpower has made...   
1    UNPqb4ThKHg  TESLA vs Uber : Will UBER Be Dead by 2030? | B...   
2    lBCiFPh87ik                        Packaging You Can't Ignore!   
3    -XLYBiMJDI0                           How can India go global?   
4    XczYinm2pQc             Smart Founders Don’t Ask… They Observe   
..           ...                                                ...   
825  nvN3cpi2Dgg                       How to never get distracted?   
826  odPwOQI_O0Q  Train your mind to never get nervous in any si...   
827  AxR6o03eKV0                           Think School Live Stream   
828  cWsxSP3KGvs  Can India's National Education Policy (NEP 202...   
829  3tVoKZ5pHMU  Indian Education System EXPOSED & How can we f...   

             published_at  view_count  like_count  comment_count  duration  
0    2025-12-03T04:53:52Z       48511 

In [20]:
df.isnull().sum()

video_id         0
title            0
published_at     0
view_count       0
like_count       0
comment_count    0
duration         0
dtype: int64

In [21]:
df.columns

Index(['video_id', 'title', 'published_at', 'view_count', 'like_count',
       'comment_count', 'duration'],
      dtype='object')

In [22]:
import re

In [23]:
def parse_iso_duration(d):
    # Handle zero-duration
    if d == 'P0D':
        return [0, 0, 0]
    
    # Regex for ISO 8601 video durations
    pattern = r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?'
    match = re.match(pattern, d)
    
    if not match:
        return [0, 0, 0]
    
    h, m, s = match.groups()
    return [int(h or 0), int(m or 0), int(s or 0)]

In [24]:
df[['hours', 'minutes', 'seconds']] = df['duration'].apply(
    lambda x: pd.Series(parse_iso_duration(x))
)

In [25]:
df.head()

,video_id,title,published_at,view_count,like_count,comment_count,duration,hours,minutes,seconds
0,X0QmjGzoW98,SHOCKING : Meesho’s Hidden Superpower has made...,2025-12-03T04:53:52Z,48511,2910,126,PT14M48S,0,14,48
1,UNPqb4ThKHg,TESLA vs Uber : Will UBER Be Dead by 2030? | B...,2025-11-27T12:08:26Z,203724,5828,329,PT17M45S,0,17,45
2,lBCiFPh87ik,Packaging You Can't Ignore!,2025-11-27T06:50:02Z,345980,8398,48,PT37S,0,0,37
3,-XLYBiMJDI0,How can India go global?,2025-11-26T11:30:00Z,34283,636,8,PT52S,0,0,52
4,XczYinm2pQc,Smart Founders Don’t Ask… They Observe,2025-11-25T12:45:11Z,34146,585,2,PT37S,0,0,37


In [26]:
df['title']

0      SHOCKING : Meesho’s Hidden Superpower has made...
1      TESLA vs Uber : Will UBER Be Dead by 2030? | B...
2                            Packaging You Can't Ignore!
3                               How can India go global?
4                 Smart Founders Don’t Ask… They Observe
                             ...                        
825                         How to never get distracted?
826    Train your mind to never get nervous in any si...
827                             Think School Live Stream
828    Can India's National Education Policy (NEP 202...
829    Indian Education System EXPOSED & How can we f...
Name: title, Length: 830, dtype: object

In [27]:
df['podcast'] = df['title'].apply(lambda x: 'Yes' if ' | IBP' in str(x) else 'No')

In [28]:
df.head()

,video_id,title,published_at,view_count,like_count,comment_count,duration,hours,minutes,seconds,podcast
0,X0QmjGzoW98,SHOCKING : Meesho’s Hidden Superpower has made...,2025-12-03T04:53:52Z,48511,2910,126,PT14M48S,0,14,48,No
1,UNPqb4ThKHg,TESLA vs Uber : Will UBER Be Dead by 2030? | B...,2025-11-27T12:08:26Z,203724,5828,329,PT17M45S,0,17,45,No
2,lBCiFPh87ik,Packaging You Can't Ignore!,2025-11-27T06:50:02Z,345980,8398,48,PT37S,0,0,37,No
3,-XLYBiMJDI0,How can India go global?,2025-11-26T11:30:00Z,34283,636,8,PT52S,0,0,52,No
4,XczYinm2pQc,Smart Founders Don’t Ask… They Observe,2025-11-25T12:45:11Z,34146,585,2,PT37S,0,0,37,No


In [29]:
df['podcast'].value_counts()

podcast
No     814
Yes     16
Name: count, dtype: int64

In [30]:
df['total_seconds'] = df['hours'] * 3600 + df['minutes'] * 60 + df['seconds']

In [31]:
df['duration_minutes'] = df['total_seconds'] / 60.0

In [32]:
df['is_short_strict'] = df['total_seconds'] <= 60

In [33]:
shorts_strict_df = df[df['is_short_strict']].copy()

In [34]:
non_shorts_df = df[~df['is_short_strict']].copy()

In [35]:
shorts_strict_df.to_csv('thinkschool_shorts.csv', index=False)
non_shorts_df.to_csv('thinkschool_non_shorts.csv', index=False)

In [36]:
df_shorts = pd.read_csv('thinkschool_shorts.csv')

In [37]:
df_shorts.shape

(310, 14)

In [38]:
df_non_shorts = pd.read_csv('thinkschool_non_shorts.csv')

In [39]:
df_non_shorts.shape

(520, 14)